<a href="https://colab.research.google.com/github/pauldebayan/SpectaGen/blob/main/SpectaGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import files
# uploaded = files.upload()


In [ ]:
!wget https://raw.githubusercontent.com/pauldebayan/SpectaGen/refs/heads/main/labels.csv
!wget https://github.com/pauldebayan/SpectaGen/raw/refs/heads/main/spectacle_dataset.zip
!unzip spectacle_dataset.zip

In [ ]:
!ls

In [ ]:
from torchvision.io import read_image
import matplotlib.pyplot as plt
import torchvision
import torch
from torch import nn
import os
import pandas as pd
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn

In [ ]:
def ImagesToDelete():
    # To check all the images is of same shape
    channel, height, width = 3, 512, 512

    imagesToDelete = []

    for i in range(1, 1000):
        img = read_image(f'./spectacle_dataset/specs{(i+1)}.jpg')

        if(img.shape[0] != channel or img.shape[1] != height or img.shape[2] != width):
            imagesToDelete.append(f'specs{(i+1)}.jpg')

    # We need to delete this images as they do not have the desired shape - [3, 256, 512]
    return imagesToDelete

print(ImagesToDelete())


In [ ]:
# Check if GPU is available and move model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 20, 2)
        self.conv2 = nn.Conv2d(20, 10, 2)
        self.conv3 = nn.Conv2d(10, 10, 2)

        self.activ = nn.LeakyReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)

        self.linear_stack = nn.Sequential(
            nn.Linear(39690, 2000),
            nn.LeakyReLU(),
            nn.Linear(2000,800),
            nn.LeakyReLU(),
            nn.Linear(800,400),
            nn.LeakyReLU(),
            nn.Linear(400, 1),
            nn.Sigmoid()
        )

    def forward(self, x):

        x = self.pool(self.activ(self.conv1(x)))
        x = self.pool(self.activ(self.conv2(x)))
        x = self.pool(self.activ(self.conv3(x)))
        x = self.dropout(x)

        x = x.view(x.size(0), -1)
        x = self.linear_stack(x)
        return x

        #return x.shape



In [ ]:
class Genearator(nn.Module):

    def __init__(self):
        super().__init__()

        self.gen_sequence = nn.Sequential(
            nn.ConvTranspose2d(8, 512, 6, stride=2),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(512, 512, 4, stride=3),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(512, 512, 6, stride=2),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(512, 512, 5, stride=2),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(),

            nn.ConvTranspose2d(512, 3, 4, stride=2),
            nn.Tanh()
        )


    def forward(self, x):
        x = x.view(-1, 8, 8, 8)  # [batch_size, channels, height, width]
        x = self.gen_sequence(x)
        return x


In [ ]:
generator = Genearator()
generator = generator.to(device)
discriminator = Discriminator()
discriminator = discriminator.to(device)

print(discriminator)
print(generator)

In [ ]:
# To get the input of the linear layer in Discriminator

img = read_image('./spectacle_dataset/specs1.jpg')
print(img.shape)
print(discriminator(img.float()))
#39690
#Will cause error because of batch size

In [ ]:
def generateNoise():
    noise = torch.randn(1, 512)
    return noise

# Conv2d: (N-K+1)/S
# ConvTranspose2d: (N-1)*S+K

In [ ]:
noise = generateNoise()
noise = noise.to(device)
img = generator(noise)

img = img.to(device)

print(img.shape)
plt.imshow(img.cpu().squeeze().detach().permute(1, 2, 0))

print(f"Fake image shape: {img.shape}")


In [ ]:
# Real Images - set dataloader
class SpectDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label


In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

dataset = SpectDataset(img_dir = 'spectacle_dataset',
                             annotations_file = 'labels.csv',
                             transform = transform)

batch_size = 8
dl = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
epochs = 10000000
lr = 0.0001

In [ ]:
loss_fn = nn.BCELoss()
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr)

In [ ]:
# Implementing GANs

for epoch in range(epochs):



    batch_counter = 1
    dis_loss_max = 0
    gen_loss_min = 100

    for real_img, real_label in dl:

        real_img = real_img.to(device)
        real_label = torch.tensor(1).float().to(device)

        noise = generateNoise().to(device)
        fake_img = generator(noise).to(device)
        fake_label = torch.tensor(0).float().to(device)


        #Generator
        optimizer_G.zero_grad()
        loss_G = loss_fn(discriminator(fake_img).squeeze(), real_label)
        loss_G.backward()
        optimizer_G.step()


        #Discriminator
        optimizer_D.zero_grad()
        fake_loss = loss_fn(discriminator(fake_img.detach()).squeeze(), fake_label)
        real_loss = loss_fn(discriminator(real_img).mean().squeeze(), real_label)
        loss_D = (fake_loss+real_loss)/2
        loss_D.backward()
        optimizer_D.step()

        gen_loss = loss_G.item()
        dis_loss = loss_D.item()

        if (gen_loss < gen_loss_min):
            gen_loss_min = gen_loss

        if (dis_loss > dis_loss_max):
            dis_loss_max = dis_loss

        print(f"Epoch{epoch+1} Batch{batch_counter}: Discriminator Loss: {dis_loss}, Max: {dis_loss_max}, Generator Loss: {gen_loss}, Min: {gen_loss_min}")
        batch_counter += 1


    if ((epoch+1)%1) == 0:
        noise = generateNoise()
        noise = noise.to(device)
        fake_img = generator(noise)
        fake_img = fake_img.to(device)
        torchvision.utils.save_image(fake_img, 'generated.jpg')
        plt.imshow(fake_img.cpu().squeeze().detach().permute(1, 2, 0))
        plt.show()
        torch.save(generator.state_dict(), 'model.pt')
        #playsound('epoch10.mp3')

    if gen_loss > 100:
        break


